# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
output_data_file = r"C:\Git\06-Python-APIs\Instructions\output_data\cities.csv"

# Import API key
from api_keys import g_key
import gmaps.datasets
gmaps.configure(api_key=g_key)
# Range of latitudes and longitudes


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities_df = pd.read_csv(r"C:\Git\python-api-challenge\output_data\cities_2.csv", encoding="utf-8")
# Drop NaN Values within dataframe
cities_df = cities_df.dropna()
cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,64.42,50.0,0.0,4.00,AR,1.635199e+09
1,1,koslan,63.4564,48.8989,28.33,87.0,100.0,0.87,RU,1.635199e+09
2,2,katsuura,35.1333,140.3000,57.22,94.0,100.0,24.70,JP,1.635199e+09
3,3,itambacuri,-18.0311,-41.6850,70.84,95.0,97.0,3.76,BR,1.635199e+09
4,5,ushuaia,-54.8000,-68.3000,33.46,93.0,40.0,21.85,AR,1.635199e+09
5,6,tateyama,34.9833,139.8667,57.13,88.0,75.0,27.63,JP,1.635199e+09
6,7,naples,40.8333,14.2500,65.28,76.0,20.0,5.99,IT,1.635199e+09
7,8,mataura,-46.1927,168.8643,62.76,70.0,89.0,1.99,NZ,1.635199e+09
8,9,albany,42.6001,-73.9662,55.94,90.0,100.0,3.27,US,1.635199e+09
9,11,baiyin,36.5583,104.2081,40.08,71.0,99.0,2.04,CN,1.635199e+09


In [15]:
len(cities_df)

569

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity"].astype(float)
#Define center location for map by finding coordinates for Bossangoa in  Central African Republic
center_location = cities_df.loc[cities_df['City'] == "bossangoa", ["Lat","Lng"]]
#Get coordinates for Bossangoa map centering
center_loc = (6.4926,17.4552)
# Plot Heatmap
fig = gmaps.figure(center=center_loc, zoom_level=2)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_df["Humidity"]),
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
#Display map center location
center_location

,Lat,Lng
254,6.4926,17.4552


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [109]:
# Narrow down the cities to specific temperature range between 70-80F, Wind less than 10mph and 0 Cloudiness
vac_cities_df = cities_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

#Find cities with ideal Max Temperature 70-80F, Wind Spped less than 10mph, and Cloudiness at 0
vac_cities_df = vac_cities_df.loc[(vac_cities_df["Max Temp"] < 80) & (vac_cities_df["Max Temp"] > 70) &\
                                  (vac_cities_df["Wind Speed"] < 10) &\
                                  (vac_cities_df["Cloudiness"] == 0)]

# Drop any empty cells and reset the index
vac_cities_df = vac_cities_df.dropna().reset_index()

vac_cities_df

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,71,79,angoche,-16.2325,39.9086,77.74,90.0,0.0,5.95,MZ,1.635199e+09
1,102,112,saint-pierre,-21.3393,55.4781,73.04,78.0,0.0,8.05,RE,1.635199e+09
2,107,118,port hedland,-20.3167,118.5667,73.45,88.0,0.0,8.05,AU,1.635199e+09
3,277,306,bhuj,23.2667,69.6667,74.59,60.0,0.0,7.23,IN,1.635199e+09
4,280,309,kot diji,27.3427,68.7083,74.14,41.0,0.0,4.85,PK,1.635199e+09
5,322,354,umm lajj,25.0213,37.2685,79.34,54.0,0.0,6.04,SA,1.635199e+09
6,388,426,sidhauli,27.2833,80.8333,71.49,72.0,0.0,5.82,IN,1.635199e+09
7,389,427,riyadh,24.6877,46.7219,74.07,40.0,0.0,8.01,SA,1.635199e+09
8,390,428,sabha,27.0377,14.4283,73.99,23.0,0.0,9.22,LY,1.635199e+09
9,443,483,esna,25.2934,32.5540,73.17,39.0,0.0,9.78,EG,1.635199e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [177]:
#this section is used just to review json file for atributes
lat = -16.2325
lng = 39.9086
params = {
    "location": f"{lat},{lng}",
    "radius": 5000,
    "types" : "hotel",
    "key": g_key
    }
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params)
resp_jsn = requested.json()
resp_jsn

{'html_attributions': [],
 'next_page_token': 'Aap_uEDeEdJeRETaPEr9-qg1oPkxhB_Z8qmwivHsYaTMp-PUa6Ju0We8ajO5VIXEoJdscMd0QnIb6zshyOo9pLgeqpLk6bDmmqKX-odHg2tPfFmDajG-hAHAJ5-qs2JIwbIqGzEk8xoeVbkwCFCW9FbqjR2cHQVMx2iyefIEz5vPGas5HqWbUSLlwW6HyMl9rHAhB1OVITh5A1xjZc-ovQV2d7IJRlN900dAELkICo_Knd4lNyZ9wb0NW5hANwb9kr7Pf5TOuVYfYFv_ywRK2h9X97EpTP80tI4De743cnZbWr5zE1kjdKZaEdoVq5C70TLce8F-UVkbzHVUxt3wiDdbpS7iRs4ngNsQjwWQZ-DYm3Xfiv4UiMOUsXgOZy4FSlHCX-RlF-TQfHlekeZ8Sa9MXdAB-kxfYFx8UUak2rxVu0KYEpFn3s-AIA',
 'results': [{'geometry': {'location': {'lat': -16.2162656, 'lng': 39.914466},
    'viewport': {'northeast': {'lat': -16.20165223043477,
      'lng': 39.92646225291706},
     'southwest': {'lat': -16.24632003867398, 'lng': 39.88517759543214}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Angoche Island',
   'photos

In [178]:
#Create hotel_df dataframe from vac_cities_df dataframe
hotel_df = vac_cities_df
#Ceate an empty hotels list that will be populated later
hotels = []
#Use for loop to cycle through hotel_df and obtain values for latitude, longitude and city
#Additionally, make search based on parameters below to find nearest hotel
for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]["Lat"]
    lng = hotel_df.loc[i]["Lng"]
    city = hotel_df.loc[i]["City"]
#Assign search parameters for gmaps
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
#Assign  url for search  
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params)
    resp_jsn = response.json()
    print(f"Retrieving Results for index: {[i]} : {city}.")
#Error Handling
    try:
        hotels.append(resp_jsn['results'][0]['name'])
    except (KeyError, IndexError):
        hotels.append("")
        print(f"Index not found: {[i]}")
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.dropna(how='any')
hotel_df


Retrieving Results for index: [0] : angoche.
Retrieving Results for index: [1] : saint-pierre.
Retrieving Results for index: [2] : port hedland.
Retrieving Results for index: [3] : bhuj.
Retrieving Results for index: [4] : kot diji.
Retrieving Results for index: [5] : umm lajj.
Retrieving Results for index: [6] : sidhauli.
Retrieving Results for index: [7] : riyadh.
Retrieving Results for index: [8] : sabha.
Retrieving Results for index: [9] : esna.
Retrieving Results for index: [10] : hukuntsi.


,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,71,79,angoche,-16.2325,39.9086,77.74,90.0,0.0,5.95,MZ,1.635199e+09,Angoche Island
1,102,112,saint-pierre,-21.3393,55.4781,73.04,78.0,0.0,8.05,RE,1.635199e+09,Saint Pierre
2,107,118,port hedland,-20.3167,118.5667,73.45,88.0,0.0,8.05,AU,1.635199e+09,Port Hedland
3,277,306,bhuj,23.2667,69.6667,74.59,60.0,0.0,7.23,IN,1.635199e+09,Bhuj
4,280,309,kot diji,27.3427,68.7083,74.14,41.0,0.0,4.85,PK,1.635199e+09,Kot Banglo
5,322,354,umm lajj,25.0213,37.2685,79.34,54.0,0.0,6.04,SA,1.635199e+09,Umluj
6,388,426,sidhauli,27.2833,80.8333,71.49,72.0,0.0,5.82,IN,1.635199e+09,Sidhauli
7,389,427,riyadh,24.6877,46.7219,74.07,40.0,0.0,8.01,SA,1.635199e+09,Riyadh
8,390,428,sabha,27.0377,14.4283,73.99,23.0,0.0,9.22,LY,1.635199e+09,Sabhā
9,443,483,esna,25.2934,32.5540,73.17,39.0,0.0,9.78,EG,1.635199e+09,Esna


In [179]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [180]:

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))